In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns
from statannotations.Annotator import Annotator

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Load lenti data for all cell types (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'
data, quantiles, stats, metadata = base.load_data(base_path, metadata_path, 'lenti')

Load application-relevant transfection data (`data2`)

In [ ]:
data2, quantiles2, stats2, metadata2 = base.load_data(base_path, metadata_path, 'application')

Load qPCR data (`data_qpcr`)

In [ ]:
data_qpcr, stats_qpcr, _ = base.load_data(base_path, metadata_path, 'qPCR')
stats_qpcr['condition'] = stats_qpcr['construct'] + '_' + stats_qpcr['dox'].astype(str)

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

# Create dicts to specify colors/markers
metadata2 = base.get_metadata(metadata_path/'construct-metadata.xlsx', 'applications')
metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
app_palette = metadata_dict2['color']
app_markers = metadata_dict2['markers']

In [ ]:
# Create palette for -/+ dox conditions (qPCR)
pal_dox = {k+'_True': v for k,v in metadata_dict2['color'].items()}
pal_none = {k+'_False': base.get_light_color(v) for k,v in metadata_dict2['color'].items()}
condition_palette = pal_dox | pal_none

In [ ]:
# Since there is no marker-only condition, save the output expression stats for untransduced cells
baseline_df = data[(data['construct'].isin(['UI','UT'])) | (data['virus_dilution']==0)].groupby(['cell','dox','exp','biorep'])['output'].apply(sp.stats.gmean).rename('output_gmean').reset_index()

### Set up figure

In [ ]:
# Set plotting context
sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc=base.rc_context)
plt.rcParams.update(base.rc_params)
scatter_kwargs = base.scatter_kwargs

# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(base.figure_width['full'], 1.5*6))
fig_gridspec = matplotlib.gridspec.GridSpec(6, 6, figure=fig, wspace=0.4, hspace=0.4, width_ratios=[1]*6,)

subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:3]),
    'B': fig.add_subfigure(fig_gridspec[0,3:]),
    'C': fig.add_subfigure(fig_gridspec[1,:4]),
    'D': fig.add_subfigure(fig_gridspec[2,:3]),
    'E': fig.add_subfigure(fig_gridspec[2,3:]),
    'F': fig.add_subfigure(fig_gridspec[3,:5]),
    'G': fig.add_subfigure(fig_gridspec[3,5:]),
    'H': fig.add_subfigure(fig_gridspec[4,:5]),
    'I': fig.add_subfigure(fig_gridspec[4,5:]),
    'J': fig.add_subfigure(fig_gridspec[5,:]),
}

# Add subpanel labels
for label, subfig in subfigures.items():
    if '2' in label: continue
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

# Save to output folder
output_path = rd.rootdir/'output'/'fig_applications-supp'/'fig_applications-supp.pdf'
fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['A']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.35, top=0.6, right=0.15)
axes = subfig.subplots(1,4, gridspec_kw=dict(wspace=0.5))

plot_df = stats2[stats2['group'].isin(['controller','base']) & (stats2['design']<=1) & (stats2['promoter']=='EF1a') & 
                 (stats2['data_group']=='iPS11_transfection')]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', ylim=(1e3,1e5))

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_std', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log', ylim=(6e3,6e5))

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='slope', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='',ylim=(0.5,1.2))

# CV
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_variation', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='CV', xlabel='', ylabel='',)

for ax in axes:
    #sns.despine(ax=ax)
    ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)


# perform statistical tests
pairs = [('CL','base'), ('CL','OL')]

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.35, top=0.6, right=0.15)
axes = subfig.subplots(1,4, gridspec_kw=dict(wspace=0.5))

plot_df = stats2[stats2['group'].isin(['controller','base']) & (stats2['design']<=1) & (stats2['promoter']=='CAG') & 
                 (stats2['data_group']=='iPS11_transfection')]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', ylim=(1e3,1e5))

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_std', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log', ylim=(6e3,6e5))

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='slope', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='',ylim=(0.5,1.2))

# CV
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_variation', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='CV', xlabel='', ylabel='',)

for ax in axes:
    sns.despine(ax=ax)
    ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.6, bottom=0.5, top=0.35, right=0)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.3))

# iPSC controls
group_order = ['base','miR','ts3','ts5']
plot_df = stats2[~stats2['group'].isin(['controller','marker']) & (stats2['promoter']=='EF1a') & (stats2['data_group']=='iPS11_transfection')].copy()
plot_df['group'] = plot_df['group'].astype(pd.api.types.CategoricalDtype(categories=group_order, ordered=True))
plot_df.sort_values(['group','ts'], inplace=True)

# shift xticks to add more space between promoter groups
buffer = 0.6
num_groups = 3
xtick_locs = [0, 1+buffer, 2+buffer] + [i+buffer*2 for i in range(3,7)]
construct_loc = {k:v for k,v in zip(plot_df['construct'].unique(), xtick_locs)}
plot_df['construct_loc'] = plot_df['construct'].replace(construct_loc)
metadata['construct_loc'] = metadata['construct'].map(construct_loc)
m = metadata.dropna()
m['construct_loc'] = m['construct_loc'].astype(str)
xlim = (-0.5, plot_df['construct_loc'].max()+0.5)

# adjust markers
m.loc[(m['miR_loc']=='CDS') | (m['ts_loc']=='3\''), 'markers'] = 'D'
m_dict = m.set_index('construct').to_dict('dict')
comb_markers = m_dict['markers']

# make xticklabels
def get_label(df):
    group = df['group'].unique()[0]
    d = df.copy()
    col_map = {'base': 'group', 'miR': 'miR', 'ts3': 'ts', 'ts5': 'ts'}
    d['label'] = d[col_map[group]]
    return d

m = m.groupby('group')[m.columns].apply(get_label).reset_index(drop=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='output_gmean', hue='construct', palette=main_palette,
                    ax=ax, marker=comb_markers[construct], **scatter_kwargs, native_scale=True)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs,)

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='output_std', hue='construct', palette=main_palette,
                    ax=ax, marker=comb_markers[construct], **scatter_kwargs, native_scale=True)
ax.set(title='Standard deviation', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs,)

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='slope', hue='construct', palette=main_palette,
                    ax=ax, marker=comb_markers[construct], **scatter_kwargs, native_scale=True)
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0.5,1.2), ylabel='', xticks=xtick_locs,)

for ax in axes:
    sns.despine(ax=ax)

    # add shaded region for miR-only constructs
    span1 = (xtick_locs[0]+(xtick_locs[1]-xtick_locs[0])/2, xtick_locs[2]+(xtick_locs[3]-xtick_locs[2])/2,)
    ax.axvspan(*span1, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    labels = dict(zip(m.drop_duplicates('construct_loc')['construct_loc'], m.drop_duplicates('construct_loc')['label']))
    ax.set_xticklabels([labels[l.get_text()].replace('.','-') for l in ax.get_xticklabels()], ha='right', rotation=45)
    ax.yaxis.set_minor_formatter(plt.NullFormatter())

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.35, top=0.6, right=0.15)
axes = subfig.subplots(1,4, gridspec_kw=dict(wspace=0.5))

plot_df = stats2[(stats2['name'].str.contains('FXN')) & (stats2['data_group']=='therapeutic_transfection')]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log',)

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_std', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log',)

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='slope', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='',)

# CV
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_variation', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='CV', xlabel='', ylabel='',)

for ax in axes:
    sns.despine(ax=ax)
    ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)
    
fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.35, top=0.6, right=0.15)
axes = subfig.subplots(1,4, gridspec_kw=dict(wspace=0.5))

plot_df = stats2[(stats2['name'].str.contains('FMRP')) & (stats2['data_group']=='therapeutic_transfection')]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log',)

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_std', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log',)

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='slope', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='',)

# CV
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_variation', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='CV', xlabel='', ylabel='',)

for ax in axes:
    sns.despine(ax=ax)
    ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)
    
fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.6, bottom=0.4, top=0.35, right=0.1)
axes = subfig.subplots(1,5, gridspec_kw=dict(width_ratios=(1,1,1,1,1.5), wspace=0.4))

plot_df = stats[(stats['cell']=='therapeutic') & (stats['dox']==1000) & (stats['name'].str.contains('FXN'))]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', )#ylim=(1e4,5e5))
sns.despine(ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_std', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log',)
sns.despine(ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='slope', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='', ylim=(0.2,1))
sns.despine(ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# CV
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_variation', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='CV', xlabel='', ylabel='',)
sns.despine(ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# CDF
ax = axes[4]
plot_df = quantiles[(quantiles['cell']=='therapeutic') & (quantiles['dox']==1000) & 
                    (quantiles['name'].str.contains('FXN')) & (quantiles['biorep']==1)]
plot_df.sort_values(['ts_num','ts_label'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

sns.kdeplot(data=plot_df, x='output', hue='construct', palette=app_palette, ax=ax,
            cumulative=True, common_norm=False, legend=False, log_scale=True)
ax.set(xticks=np.logspace(2,6,5), ylabel='', xlim=(1e1,2e4), title='CDF')
sns.despine(ax=ax)
ax.minorticks_off()
ax.grid(zorder=-1, color=base.get_light_color(base.get_light_color(base.colors['gray'])), which='both',alpha=0.7)

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['G']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.4, top=0.35, right=0.05)
axes = subfig.subplots(1,1,)

plot_df = stats[(stats['cell']=='therapeutic') & (stats['dox']==0) & (stats['name'].str.contains('FXN'))]

# stat gmean
ax = axes 
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                    ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', ylim=(1e1,2e2))
baseline = baseline_df.loc[(baseline_df['cell']=='iPS11') & (baseline_df['dox']==0), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['H']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.6, bottom=0.4, top=0.35, right=0.1)
axes = subfig.subplots(1,5, gridspec_kw=dict(width_ratios=(1,1,1,1,1.5), wspace=0.4))

plot_df = stats[(stats['cell']=='therapeutic') & (stats['dox']==1000) & (stats['name'].str.contains('FMRP'))]

# mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', )#ylim=(1e4,5e5))
sns.despine(ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_std', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlabel='', ylabel='', yscale='log', )#ylim=(1e4,5e5))
sns.despine(ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='slope', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlabel='', ylabel='', )#ylim=(0.5,1.4))
sns.despine(ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# CV
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_variation', hue='construct', palette=app_palette,
                  ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='CV', xlabel='', ylabel='', yticks=(0.8,1,1.2,1.4))
sns.despine(ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

# CDF
ax = axes[4]
plot_df = quantiles[(quantiles['cell']=='therapeutic') & (quantiles['dox']==1000) & 
                    (quantiles['name'].str.contains('FMRP')) & (quantiles['biorep']==1)]
plot_df.sort_values(['ts_num','ts_label'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

sns.kdeplot(data=plot_df, x='output', hue='construct', palette=app_palette, ax=ax,
            cumulative=True, common_norm=False, legend=False, log_scale=True)
ax.set(xticks=np.logspace(2,6,5), ylabel='', xlim=(1e2,1e5), title='CDF')
sns.despine(ax=ax)
ax.minorticks_off()
ax.grid(zorder=-1, color=base.get_light_color(base.get_light_color(base.colors['gray'])), which='both',alpha=0.7)

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['I']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.4, top=0.35, right=0.05)
axes = subfig.subplots(1,1,)

plot_df = stats[(stats['cell']=='therapeutic') & (stats['dox']==0) & (stats['name'].str.contains('FMRP'))]

# stat gmean
ax = axes #[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_label', y='output_gmean', hue='construct', palette=app_palette,
                    ax=ax, marker=app_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlabel='', ylabel='', yscale='log', ylim=(1e1,2e2))
baseline = baseline_df.loc[(baseline_df['cell']=='iPS11') & (baseline_df['dox']==0), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')
ax.set_xticklabels(ax.get_xticklabels(), ha='right', rotation=45)

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['J']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.6, bottom=0.4, top=0.35, right=0.05)
axes = subfig.subplots(1,7, gridspec_kw=dict(wspace=0.5))

primers_list = ['FXN', 'mRuby2-1', 'mRuby2-2', 'FMRP-1', 'FMRP-2', 'EGFP-1', 'EGFP-2']
for ax, primers in zip(axes, primers_list):
    plot_df = stats_qpcr[(stats_qpcr['primers']==primers) & ~(stats_qpcr['construct'].isin(['no-RT','no-cDNA','UI']))]
    
    for construct, group in plot_df.groupby('construct'):
        # # Plot -dox as marker outline
        # scatter_kwargs_updated = scatter_kwargs.copy()
        # scatter_kwargs_updated['edgecolor'] = app_palette[construct]
        # scatter_kwargs_updated['s'] = scatter_kwargs_updated['s'] - scatter_kwargs_updated['linewidth']*2
        # sns.stripplot(data=group[~group['dox']], x='ts_label', y='norm_expression', ax=ax, color='white',
        #               marker=main_markers[construct], **scatter_kwargs_updated)
        
        # Plot +dox as normal
        sns.stripplot(data=group[group['dox']], x='ts_label', y='relative_expression', ax=ax, hue='construct', palette=app_palette,
                      marker=main_markers[construct], **scatter_kwargs)

    ax.set(xlabel='', ylabel='', title=primers)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    #baseline = stats_qpcr.loc[(stats_qpcr['primers']==primers) & (stats_qpcr['construct']=='UI'), 'relative_expression'].median()
    ax.axhline(1, c='black', ls=':', zorder=0)
    if primers in primers_list[:3]: ax.set_title(ax.get_title(), color=base.colors['orange'])
    else: ax.set_title(ax.get_title(), color=base.colors['red'])

axes[0].set(ylabel='Expression, $2^{-\Delta\Delta Ct}$')

fig.savefig(rd.outfile(output_path))

In [ ]:
# Save to OneDrive
fig.savefig(rd.outfile(rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'links'/'fig_applications-supp.pdf'))